In [77]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from torch.autograd import Variable
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

In [102]:
EPOCHS_ARRAY = [1,5,10,15,30]
BATCH_SIZE_ARRAY = [1,2,4,8,10,16,24]
TRAIN_SIZE_ARRAY = [200,400,600,800,-1]

EPOCHS = EPOCHS_ARRAY[0]
TRAIN_BATCH_SIZE = BATCH_SIZE_ARRAY[4]
TRAIN_SIZE = TRAIN_SIZE_ARRAY[0]

BATCH_SIZE = 10

LEARNING_RATE = 0.0003
TRAIN_DATAPATH = 'chest_xray\\train'
VAL_DATAPATH = 'chest_xray\\val'
TEST_DATAPATH = 'chest_xray\\test'
TRAIN_TRANSFORM_IMG = transforms.Compose([
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees = 15),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])

VAL_TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATAPATH, transform=TRAIN_TRANSFORM_IMG)
train_data_loader = torch.utils.data.DataLoader(train_data,batch_size=TRAIN_BATCH_SIZE,shuffle=True,num_workers=4)
val_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=VAL_TRANSFORM_IMG)
val_data_loader = torch.utils.data.DataLoader(val_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATAPATH, transform=VAL_TRANSFORM_IMG)
test_data_loader = torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=False,num_workers=4)

print_row = []
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.figure(figsize=(20,20))
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))


# dataiter = iter(train_data_loader)
# images, labels = dataiter.next()
# # show images
# imshow(torchvision.utils.make_grid(images))

In [25]:
def Net(x):
    if x == 'vgg19':
        vgg19 = models.vgg19(True)
        for param in vgg19.parameters():
            param.requires_grad = False
        vgg19.classifier[6] = nn.Sequential(nn.Linear(4096, 256), 
                              nn.ReLU(), 
                              nn.Dropout(0.4),
                              nn.Linear(256, 2),                   
                              nn.LogSoftmax(dim=1))
        return vgg19
    
    if x == 'res18':
        res18 = models.resnet18(True)
        for param in res18.parameters():
            param.requires_grad = False
        res18.fc = nn.Sequential(nn.Linear(512, 256), 
                              nn.ReLU(), 
                              nn.Dropout(0.4),
                              nn.Linear(256, 2),                   
                              nn.LogSoftmax(dim=1))
        return res18

In [8]:
print("Number of Training Examples: ", len(train_data))
print("Number of Test Examples: ", len(test_data))
print("Number of Validation Examples: ", len(val_data))
print("Detected Classes are: ", train_data.class_to_idx)
train_iter = iter(train_data_loader)
images, labels_ = train_iter.next()
print("Image Shape on Batch size = {} ".format(images.size()))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

vgg19 = Net('vgg19')
res18 = Net('res18')
    
vgg19.to(device)
res18.to(device)
weight = torch.tensor([0.75,0.25])
weight = weight.to(device)
criterion = nn.NLLLoss(weight = weight)
_paramsvgg19 = filter(lambda p: p.requires_grad, vgg19.parameters())
optimizervgg19 = optim.Adam(_paramsvgg19, lr=LEARNING_RATE, weight_decay=1e-5)
_paramsvgg19 = filter(lambda p: p.requires_grad, res18.parameters())
optimizerres18 = optim.Adam(_paramsvgg19, lr=LEARNING_RATE, weight_decay=1e-5)

Number of Training Examples:  5216
Number of Test Examples:  624
Number of Valid Examples:  624
Detected Classes are:  {'NORMAL': 0, 'PNEUMONIA': 1}
Image Shape on Batch size = torch.Size([10, 3, 224, 224]) 
cuda:0


In [26]:
def trainModel(model, optimizer):
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []

    for epoch in range(EPOCHS):
        start = time.time()
        print("EPOCH: ",epoch)
        correct = 0 
        iterations = 0
        iter_loss = 0.0

        model.train()

        for i,data in enumerate(train_data_loader,0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            iter_loss+=loss.item()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()
            iterations += 1
            if(iterations == TRAIN_SIZE):
                break

        train_loss.append(iter_loss/iterations)
        # Record the training accuracy
        train_accuracy.append((100 * correct / len(train_data)))

        loss = 0.0
        correct = 0
        iterations = 0
        model.eval()

        for i, data in enumerate(val_data_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)     
            loss = criterion(outputs, labels)
            loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum()

            iterations+=1

        test_loss.append(loss/iterations)
        # Record the Testing accuracy
        test_accuracy.append((100 * correct / len(val_data)))
        stop = time.time()
        print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}, Validation Loss: {:.3f}, Validation Acc: {:.3f}, Time: {:.3f}s'
               .format(epoch+1, EPOCHS, train_loss[-1], train_accuracy[-1], test_loss[-1], test_accuracy[-1], stop-start))

        row = [epoch+1,train_loss[-1],train_accuracy[-1].item(),test_loss[-1].item(),test_accuracy[-1].item(),stop-start]
        print_row.append(row)
        
    predictions = testModel(model)
    return predictions

In [90]:
def testModel(model):
    correct = 0
    total = 0
    classes =['NORMAL','PNEUMONIA']
    confusion_matrix = torch.zeros(2, 2)
    prediction_list = []
    correct_0 = 0
    correct_1 = 0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    with torch.no_grad():
        for data in test_data_loader:
            images,labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            print(predicted)
            _ = (predicted.cpu())
            prediction_list.append(_.numpy())
            total += labels.size(0)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
            correct+=(predicted == labels).sum().item()
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    acc = 100*correct/total
    print(correct)
    print(total)
    print('Testing accuracy: ' + str(acc))
    print(confusion_matrix)

    for i in range(2):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
    acc0 = 100 * class_correct[0] / class_total[0]
    acc1 = 100 * class_correct[1] / class_total[1]
    print_row_end = []
    row_end = [acc,acc0,acc1]
    print_row_end.append(row_end)
    return prediction_list

In [103]:
predvgg = trainModel(vgg19, optimizervgg19)
pvgg = np.hstack(predvgg)
print(pvgg.shape)
pvgg = pvgg.reshape([-1,1])

EPOCH:  0
Epoch 1/1, Training Loss: 0.302, Training Accuracy: 33.000, Validation Loss: 0.018, Validation Acc: 80.000, Time: 39.135s
tensor([0, 0, 0, 1, 0, 0, 0, 1, 1, 0], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 0, 1], device='cuda:0')
tensor([1, 0, 1, 0, 1, 0, 0, 0, 1, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], device='cuda:0')
tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0, 1, 0, 0, 0, 0, 0, 1, 0, 1], device='cuda:0')
tensor([0, 0, 1, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')
tensor([0, 0, 0, 1, 0, 0, 0, 1, 0, 0], device='cuda:0')
tensor([0, 0, 0, 1, 0, 0, 0, 1, 1, 1], device='cuda:0')
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
tensor([0, 1, 0, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')
tensor([0, 1, 0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0, 1, 1, 1, 1, 0, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 1,

In [101]:
predres = trainModel(res18, optimizerres18)
pres = np.hstack(predres)
print(pres.shape)
pres = pres.reshape([-1,1])

EPOCH:  0
Epoch 1/1, Training Loss: 0.241, Training Accuracy: 34.000, Validation Loss: 0.002, Validation Acc: 84.000, Time: 24.018s
tensor([0, 0, 1, 1, 0, 0, 1, 1, 0, 1], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 0, 1, 1, 0], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 0, 1, 0, 1, 1, 1, 1, 1, 0], device='cuda:0')
tensor([0, 1, 1, 0, 0, 1, 0, 1, 1, 1], device='cuda:0')
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1], device='cuda:0')
tensor([1, 1, 0, 1, 0, 0, 1, 1, 1, 1], device='cuda:0')
tensor([1, 0, 0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([0, 1, 1, 1, 0, 1, 0,

In [111]:
mean_ans = (pvgg)
correct = 0
total = 0
predicted = np.floor(mean_ans)
# print(predicted)
for i,data in enumerate(test_data):
    images,labels = data[0], data[1]
    total += 1
    correct+=(predicted[i] == labels).sum().item()

print(correct/total)

0.8060897435897436


In [ ]:
filename = "vgg19+base_"+str(TRAIN_BATCH_SIZE)+"_"+str(TRAIN_SIZE)+"_"+str(EPOCHS)
heading = []
footer = []
heading.append(["Epoch Number", 'Training Loss', 'Training Accuracy','Validation Loss','Validation Acc', 'Time' ])
footer.append(['Testing Accuracy','Accuracy of Normal', 'Accuracy of Pneumonia'])
with open('Save/'+filename+'.csv', mode='w',newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(heading)
    writer.writerows(print_row)
    writer.writerows(footer)
    writer.writerows(print_row_end)
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [ ]:
print("PyTorch version: ", torch.__version__ )
print("CUDA available: ", torch.cuda.is_available())
print("CUDA version: ", torch.version.cuda)
print(filename)

In [ ]:
torch.cuda.empty_cache()